In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import csv

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/cifar10-comp"))

# Any results you write to the current directory are saved as output.

In [ ]:
#load images
data_x = np.load("../input/cifar10-comp/train_images.npy")

In [ ]:
data_x.shape

In [ ]:
# make the channals tha last dim
data_x = data_x.transpose(0,3,2,1)

In [ ]:
print(data_x.shape)
plt.imshow(data_x[20], interpolation='nearest')
plt.show()

In [ ]:
# categories of the classification
labels_i_s = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
labels_s_i = {cat:ind for ind, cat in enumerate(labels_i_s)}

In [ ]:
labels_s_i

In [ ]:
# laod labels
data_y = np.genfromtxt("../input/cifar10-comp/train_labels.csv", delimiter=',', dtype= str, skip_header=1)

In [ ]:
data_y = data_y[:,1]

In [ ]:
data_y[0]

In [ ]:
# convert string labels to numeric
labels_num = np.zeros(data_y.shape, dtype=int)
for i, label in enumerate(data_y):
    labels_num[i] = labels_s_i[label]
print(labels_num[0])

In [ ]:
# convert numeric labels to one hot vector
one_hot_encoding = np.eye(10)[labels_num]
print(one_hot_encoding[0,:])

In [ ]:
#normalize the images
data_x = data_x/255

In [ ]:
print(np.min(data_x))
print(np.max(data_x))

In [ ]:
# randomly shuffil the data
idx = np.random.permutation(data_y.shape[0])
data_x,one_hot_encoding = data_x[idx], one_hot_encoding[idx]

In [ ]:
print(one_hot_encoding[21])
plt.imshow(data_x[21], interpolation='nearest')
plt.show()

In [ ]:
# divide the data set into traning and validation
X_train = data_x
X_valid = data_x[45000:]
y_train = one_hot_encoding
y_valid = one_hot_encoding[45000:]
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

In [ ]:
# CONV Layer 
def Conv(input_data, num_input_chan, filter_size, num_filters, bias_init, name):
    with tf.variable_scope(name) as scope:
        # creat conv layer
        #regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
        layer = tf.layers.conv2d(inputs=input_data, filters=num_filters, kernel_size = [filter_size, filter_size]
                                 , strides=[1, 1], padding='same', activation = "relu", use_bias = True)
        return layer 
    
'''
def Conv(input_data, num_input_chan, filter_size, num_filters, stride, name, pad = 'SAME'):
    with tf.variable_scope(name) as scope:

        # Create filters
        filters = tf.get_variable('weghits', shape = [filter_size, filter_size, num_input_chan, num_filters])

        # Create new biases
        biases = tf.get_variable('bias', shape=[num_filters])

        # creat conv layer
        layer = tf.nn.conv2d(input=input_data, filter=filters, strides=[1, stride, stride, 1], padding=pad)

        # Add bias 
        layer += biases

        #Apply Activation
        relu = tf.nn.relu(layer)
        return layer 
    '''

In [ ]:
# MAXPOOL Layer
def Max_Pool(input_data, kernal_size, stride_size, name):
    with tf.variable_scope(name) as scope:
        layer = tf.layers.max_pooling2d(input_data, pool_size=[kernal_size, kernal_size], 
                               strides=[stride_size, stride_size],  padding="valid")
        return layer

'''
def Max_Pool(input_data, kernal_size, stride_size, name):
    with tf.variable_scope(name) as scope:
        layer = tf.nn.max_pool(input_data, ksize=[1, kernal_size, kernal_size, 1], 
                               strides=[1, stride_size, stride_size, 1],  padding="VALID")
        return layer'''

In [ ]:
'''def FC(input_data, num_input, num_output, name):
    with tf.variable_scope(name) as scope:
        #intialize the weights 
        weights = tf.get_variable('weghits',shape = [num_input, num_output])
        #intialize the bias 
        biases = tf.get_variable('bias', shape=[num_output])
        # output Layer
        layer =  tf.matmul(input_data, weights) + biases
        return layer
'''    
def FC(input_data, num_input, num_output, name):
    with tf.variable_scope(name) as scope:
        layer = tf.layers.dense(input_data, num_output)
        return layer

In [ ]:
'''#Network Archecture (ALexNet) 
#Faield

#palceholder for input data
x = tf.placeholder(tf.float32, shape=[None,32,32,3])
#place holder 
y = tf.placeholder(tf.float32, shape=[None, 10])
y_cls = tf.argmax(y, dimension=1)

# first Layer (removed)
conv1 = Conv(x, 3, 11, 96, 4, "conv1")
norm1 = tf.nn.local_response_normalization(conv1, depth_radius = 2, alpha = 1e-05, beta = 0.75, bias = 1.0, name = "norm1")
max_pool1 = Max_Pool(norm1, 3, 2, "max_pool1")
# 2d layer
conv2 = Conv(x, 3, 5, 256, 1, "conv2")
norm2 = tf.nn.local_response_normalization(conv2, depth_radius = 2, alpha = 1e-05, beta = 0.75, bias = 1.0, name = "norm2")
max_pool2 = Max_Pool(norm2, 3, 2, "max_pool2")

# 3rd layer
conv3 = Conv(max_pool2, 256, 3, 384, 1, "conv3")

# 4th layer
conv4 = Conv(conv3, 384, 3, 384, 1, "conv4")

# 5th layer
conv5 = Conv(conv4, 384, 3, 256, 1, "conv5")
max_pool5 = Max_Pool(conv5, 3, 2, "max_pool5")

# Flatten the Max Layer
flatten = tf.reshape(max_pool5, shape = [-1, max_pool5.shape[1]*max_pool5.shape[2]*max_pool5.shape[3]])

#print(flatten.shape)

# 6th layer
fc1 = FC(flatten, int(flatten.shape[0]), 4096, "fc1")
relu_fc1 = tf.nn.relu(fc1)
dropout_fc1 = tf.nn.dropout(relu_fc1, 0.5)

# 7th layer
fc2 = FC(dropout_fc1, 4096, 4096, "fc2")
relu_fc2 = tf.nn.relu(fc2)
dropout_fc2 = tf.nn.dropout(relu_fc2, 0.5)

# 7th layer
fc3 = FC(dropout_fc1, 4096, 10, "fc3")'''

In [ ]:
# archecture 2

x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
y_cls = tf.argmax(y, dimension=1)

conv1 = Conv(x, 3, 3, 32, 0.0, "conv1")
norm1 = tf.layers.batch_normalization(conv1)
relu1 = tf.nn.relu(norm1)

conv2 = Conv(relu1, 32, 3, 64, 0.0, "conv2")
norm2 = tf.layers.batch_normalization(conv2)
relu2 = tf.nn.relu(norm2)

max_pool1 = Max_Pool(relu2, 2, 2, "max_pool1")

conv3 = Conv(max_pool1, 64, 3, 128, 0.0, "conv3")
norm3 = tf.layers.batch_normalization(conv3)
relu3 = tf.nn.relu(norm3)

max_pool2 = Max_Pool(relu3, 2, 2, "max_pool2")


conv4 = Conv(max_pool2, 128, 3, 256, 0.0, "conv4")
norm4 = tf.layers.batch_normalization(conv4)
relu4 = tf.nn.relu(norm4)

max_pool3 = Max_Pool(relu4, 2, 2, "max_pool3")

flat = tf.contrib.layers.flatten(norm4)

fc1 = FC(flat, (int)(flat.shape[1]), 1024,"fc1")
fc1_relu = tf.nn.relu(fc1)
fc1_drop = tf.nn.dropout(fc1_relu, 0.7)
norm_fc1 = tf.layers.batch_normalization(fc1_drop)

fc2 = FC(fc1_drop, 1024, 512,"fc2")
fc2_relu = tf.nn.relu(fc2)
fc2_drop = tf.nn.dropout(fc2_relu, 0.7)
norm_fc2 = tf.layers.batch_normalization(fc2_drop)

fc3 = FC(fc2_drop, 512, 256,"fc3")
fc3_relu = tf.nn.relu(fc3)
fc3_drop = tf.nn.dropout(fc3_relu, 0.7)
norm_fc3 = tf.layers.batch_normalization(fc3_drop)

fc4 = FC(norm_fc3, 256, 10,"fc4")

In [ ]:
# the prediction 
with tf.variable_scope("Softmax") as scope:
    y_pred = tf.nn.softmax(fc4)
    y_pred_cls = tf.argmax(y_pred, dimension=1)

In [ ]:
# calculate the loss
#L2_loss = tf.losses.get_regularization_loss()
loss = tf.nn.softmax_cross_entropy_with_logits(logits=fc4, labels=y)
#calculate cost
cost = tf.reduce_mean(loss)

In [ ]:
# set optimizer 
learning_rate = tf.placeholder(tf.float32, shape=[])
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
# accurcy
correct_pred = tf.equal(y_cls, y_pred_cls)
accurcy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# runing variables
num_epoch = 100
num_patch = 128
train_accurses = []
valid_accurses = []

In [ ]:
test_data = np.load("../input/cifar10-comp/test_images.npy")
test_data = test_data.transpose(0,3,2,1)
test_data = test_data / 255
print(test_data.shape)

In [ ]:
init_lr = .001
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, num_epoch):
        start_time = time.time()
        train_acc = 0
        #random shuffel the training data
        idx = np.random.permutation(X_train.shape[0])
        X_train,y_train = X_train[idx], y_train[idx]
        #iterate for batches
        for step in range(int(X_train.shape[0]/num_patch)):
            #print(str(step*num_patch) +"  "+str((step+1)*num_patch))
            x_batch, y_batch = X_train[step*num_patch:(step+1)*num_patch], y_train[step*num_patch:(step+1)*num_patch] 
            # update varib
            sess.run(optimizer, feed_dict = {x: x_batch, y: y_batch, learning_rate: init_lr})
            train_acc += sess.run(accurcy, feed_dict = {x: x_batch, y: y_batch, learning_rate: init_lr})
        end_time = time.time()
        train_acc /= int(X_train.shape[0]/num_patch)
        train_accurses.append(train_acc)
        valid_acc = sess.run(accurcy, feed_dict = {x: X_valid, y: y_valid})
        valid_accurses.append(valid_acc)
        print("Epoch "+str(epoch+1)+" completed : Time usage "+str(int(end_time-start_time))+" seconds")
        print("\tAccuracy:")
        print ("\t- Training Accuracy:\t{}".format(train_acc))
        print ("\t- Validation Accuracy:\t{}".format(valid_acc))
        if(epoch%10==0):  init_lr /= 2
            
            
    # test 
    labelsss = []
    for i in test_data:
        i =  i.reshape(1,32,32,3)
        test_labels = sess.run(y_pred_cls, {x: i})
        labelsss.append((int)(test_labels))
        

In [ ]:
labelsss[3]

In [ ]:
with open('test_labels.csv', mode='w') as test:
    test_writer = csv.writer(test, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    test_writer.writerow(['Id', 'Category'])
    for inx,label in enumerate(labelsss):
        test_writer.writerow([inx+1, labels_i_s[label]])

In [ ]:
!ls